# PREPARANDO LA DATA

In [ ]:
# IMPORTAMOS LAS LIBRERIAS 

import os 
import datetime 

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

In [ ]:
# CARGAMOS EL DATASET
data = pd.read_csv('Life Expectancy Data.csv')
data

In [ ]:
# VEMOS LOS DATOS FALTANTES
data.isna().sum()

In [ ]:
# ELIMINAMOS LA DATA FALTANTE
data.dropna(inplace=True)

In [ ]:
# VERIFICAMOS QUE SE HAYA ELIMINADO LA DATA FALTANTE
data.isna().sum()

In [ ]:
categorical_col = data.select_dtypes(exclude='number')
categorical_col

In [ ]:
# PODEMOS ELIMIMAR LA COLUMNA data['Country']
data.drop('Country',axis=1,inplace=True)

In [ ]:
# CONVERTIMOS LA COLUMNA data['status'] EN NUMERICO
status_dummy = pd.get_dummies(data['Status'])
status_dummy

In [ ]:
# ELIMINAMOS LA DATA CATEGORICA 
data.drop('Status',axis=1,inplace=True)

In [ ]:
# AGREGAMOS LA DATA DUMMY
data['develop'] = status_dummy['Developed']

In [ ]:
data.describe().T

In [ ]:
# ESTANDARIZAMOS NUESTRA DATA CON StandardScaler()

s_scaler = StandardScaler()
scaled_data = s_scaler.fit_transform(data)

scaled_data = pd.DataFrame(scaled_data,
columns=data.columns,
index=data.index)

In [ ]:
scaled_data.describe().T

In [ ]:
target = data['Life expectancy ']
features = scaled_data.drop('Life expectancy ',axis=1).values

In [ ]:
# DIVIDIMOS LA DATA EN TRAIN_TEST
from sklearn.model_selection import train_test_split

x_train,x_test, y_train, y_test = train_test_split(features,target,
test_size=0.2,
random_state=1)

In [ ]:
x_train.shape

# DEFINIENDO LOS MODELOS 

In [ ]:
def single_layer_model():
    '''
    Red Neuronal de una sola copa
    - Optimer : ADAptive Moment (ADAM)
    - Metrics : Mean Square Erro (mse), 
                Mean Absolute Error (mae)
    '''

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, input_shape=(x_train.shape[1],), activation='sigmoid'),
        tf.keras.layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
    
    model.compile(loss='mse',
    metrics=['mse','mae'],
    optimizer=optimizer)

    return model 


In [ ]:
model = single_layer_model()

# PARA VER UN RESUMEN DE NUESTRO MODELO
model.summary()

In [ ]:
# PARA VER DE MANERA VISUAL NUESTRO MODELO
tf.keras.utils.plot_model(model)

In [ ]:
num_epochs = 100
training_history = model.fit(x_train,
                            y_train,
                            epochs=num_epochs,
                            validation_split=0.2,
                            verbose=True)

In [ ]:
# HACEMOS Y_PRED
y_pred = model.predict(x_test)

In [ ]:
# HALLAMOS R2_SCORE
r2 = r2_score(y_test, y_pred)
r2

In [ ]:
# VISUALIZAMOS Y_TEST AL LADO DE Y_PRED
pd.DataFrame({'y_test':y_test.values.flatten(),
'y_perd':y_pred.flatten(),
'error':abs(y_test.values.flatten()-y_pred.flatten())}
).sample(15)

In [ ]:
# PARA VER LA CORRELACIÓN DE FORMA MÁS VISUAL 
plt.figure(figsize=(10,10))
plt.scatter(y_test,y_pred)
plt.show()

# MULTIPLE LAYERS MODEL

In [ ]:
# MULTIPLE LAYERS MODEL
def multiple_layers_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32,input_shape=(x_train.shape[1],), activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(4,activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

    model.compile(loss='mse',
    metrics=['mae','mse'],
    optimizer=optimizer)

    return model

In [ ]:
model = multiple_layers_model()

model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
!rm -rf seq_logs
!ls -l

In [ ]:
# PARA VISUALIZAR LOS LOGS EN UN TENSORBOARD

logdir = os.path.join("seq_logs",datetime.datetime.now().strftime("%y%m%d-%H%m%S"))

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:

model.fit(x_train,y_train,
            epochs=100,
            validation_split=0.2,
            batch_size = 100,
            callbacks = [tensorboard_callback])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir seq_logs --port 6050

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
y_pred = model.predict(x_test)

r2 = r2_score(y_test,y_pred)
r2

In [ ]:
# VISUALIZAMOS Y_TEST AL LADO DE Y_PRED
pd.DataFrame({'y_test':y_test.values.flatten(),
'y_perd':y_pred.flatten(),
'error':abs(y_test.values.flatten()-y_pred.flatten())}
).sample(15)

# MODEL WITH STOCASTIC GRADIENT DISCENT (SGD)

In [ ]:
del model_sgd

In [ ]:


def model_with_sgd():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, input_shape=(x_train.shape[1],), activation='relu'),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

    model.compile(loss='mse',
                    metrics=['mae','mse'],
                    optimizer=optimizer)

    return model

In [ ]:
model_sgd = model_with_sgd()

In [ ]:
tf.keras.utils.plot_model(model_sgd,show_shapes=True )

In [ ]:
num_epochs = 100
training_history = model_sgd.fit(x_train,
                            y_train,
                            epochs=num_epochs,
                            validation_split=0.2,
                            batch_size=100)

In [ ]:
model_sgd.evaluate(x_test,y_test)

In [ ]:
y_pred_sgd = model_sgd.predict(x_test)

In [ ]:
r2_sgd = r2_score(y_test,y_pred_sgd)
r2_sgd

# RMSPROP

In [ ]:
def model_with_rmsprop():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, input_shape=(x_train.shape[1],), activation='elu'),
        tf.keras.layers.Dense(16, activation='elu'),
        tf.keras.layers.Dense(4, activation='elu'),
        tf.keras.layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

    model.compile(loss='mse',
                    metrics=['mae','mse'],
                    optimizer=optimizer)

    return model

In [ ]:
model_rmsprop = model_with_rmsprop()

In [ ]:
num_epochs = 100
training_history = model_rmsprop.fit(x_train,
                            y_train,
                            epochs=num_epochs,
                            validation_split=0.2,
                            batch_size=100)

In [ ]:
model_rmsprop.evaluate(x_test,y_test)

In [ ]:
y_pred_rmsprop = model_rmsprop.predict(x_test)

In [ ]:
r2_rmsprop = r2_score(y_test,y_pred_rmsprop)
r2_rmsprop